In [1]:
import pandas as pd
import os
import numpy as np
import json
import re
root_folder = "/home/trungdc/unimelb/MAST30024/asm/project 1/"
data_dir = os.path.join(root_folder, "Data/Weather")


# Load dataset from JSON to CSV

In [2]:
def process_monthly_json(JSON_Object : json):
    """
    Process monthly json object and store into list of dictionaries    
    """
    output = []
    # JSON_Object = json.loads(JSON_Object)
    for station in JSON_Object.keys():
        for day in JSON_Object[station]:
            day["station"] = station
        output += JSON_Object[station]

    return output

with open(os.path.join(data_dir, "01-2018 to 01-2019.txt")) as infile:
    s = infile.read()
    s = s.replace("\'", "\"")
    processed  = re.split("}{", s)

    df = []
    count = 1
    for month in processed:
        if count == 1:
            month += "}"
        elif count == 24:
            month = "{" + month
        else:
            month = "{" + month + "}"
        count += 1
        monthly_data = process_monthly_json(json.loads(month))
        df.extend(monthly_data)
    pd.DataFrame(df).to_csv(os.path.join(data_dir, "Weather.csv" ))
infile.close()

# Preprocessing data

## Filter irrelevant weather station & Detecting nan value in columns

In [3]:
df = pd.read_csv(os.path.join(data_dir, "Weather processed.csv" ))
print("All station in NY")
df["station"].unique()

All station in NY


array(['Bridgeport Area', 'Islip Area', 'NY-LaGuardia AP Area',
       'NY-Kennedy AP Area', 'NY-Central Park Area', 'Newark Area',
       'Caldwell, NJ', 'Canoe Brook, NJ', 'Charlotteburg, NJ',
       'Harrison, NJ', 'Newark Airport, NJ', 'Teterboro, NJ',
       'Wanaque, NJ', 'Baiting Hollow, NY', 'Bridgehampton, NY',
       'Centerport, NY', 'Farmingdale, NY', 'Islip, NY',
       'Molly CERCOM, NY', 'Montauk, NY', 'Montgomery, NY',
       'Mount Sinai, NY', 'NY Central Park, NY', 'New York JFK, NY',
       'New York LGA, NY', 'Orient Point, NY', 'Port Jervis, NY',
       'Riverhead, NY', 'Shirley, NY', 'Shrub Oak, NY', 'Syosset, NY',
       'Upton/WFO OKX, NY', 'Westhampton, NY', 'West Point, NY',
       'White Plains, NY', 'Ansonia 1 NE, CT', 'Bridgeport Arpt, CT',
       'Bridgeport Co-op, CT', 'Danbury Arpt, CT', 'Danbury Co-op, CT',
       'Groton Arpt, CT', 'Groton Co-op, CT', 'Guilford, CT',
       'Meriden Arpt, CT', 'Mount Carmel, CT', 'New Haven, CT',
       'Norwich Pub Ut

In [4]:
chosen_station = ["NY-Kennedy AP Area", "NY-LaGuardia AP Area", "Newark Area", "Teterboro, NJ","NY-Central Park Area", "New York JFK, NY"]
boolean_series = df.station.isin(chosen_station)
filter_df = df[boolean_series]
filter_df.drop_duplicates()
filter_df.head()

,Unnamed: 0,date,tempMax,tempMin,tempAvg,tempDeparture,hdd,cdd,precipitation,newSnow,snowDepth,station
62,62,2018-01-01,19,8,13.5,-22.6,51,0,0.00,0.0,0,NY-LaGuardia AP Area
63,63,2018-01-02,26,14,20.0,-15.9,45,0,0.00,0.0,0,NY-LaGuardia AP Area
64,64,2018-01-03,30,17,23.5,-12.2,41,0,0.00,0.0,0,NY-LaGuardia AP Area
65,65,2018-01-04,29,20,24.5,-11.0,40,0,0.51,7.4,1,NY-LaGuardia AP Area
66,66,2018-01-05,20,10,15.0,-20.4,50,0,0.00,0.0,7,NY-LaGuardia AP Area


In [5]:
for column in ['tempMax', 'tempMin', 'tempAvg', 'tempDeparture', 'hdd', 'cdd', 'precipitation', 'newSnow', 'snowDepth']:
   unique_values = filter_df[column].unique()
   print(f"Nonstring value in column {column}")
   for unique_value in unique_values:
      try:
         float(unique_value)
      except ValueError:
         print(unique_value)

Nonstring value in column tempMax
M
Nonstring value in column tempMin
M
Nonstring value in column tempAvg
M
Nonstring value in column tempDeparture
M
Nonstring value in column hdd
M
Nonstring value in column cdd
M
Nonstring value in column precipitation
T
M
Nonstring value in column newSnow
T
M
Nonstring value in column snowDepth
T
M


In [6]:
df = pd.read_csv(os.path.join(data_dir, "Weather processed.csv" ), na_values = ['NaN','M', 'T', 'nan'])
chosen_station = ["NY-Kennedy AP Area", "NY-LaGuardia AP Area", "Newark Area", "Teterboro, NJ","NY-Central Park Area", "New York JFK, NY"]
boolean_series = df.station.isin(chosen_station)
filter_df = df[boolean_series] # remove irrelevant station
filter_df.drop_duplicates() # remove duplicates row
# convert column into float
for column in ['tempMax', 'tempMin', 'tempAvg', 'tempDeparture', 'hdd', 'cdd', 'precipitation', 'newSnow', 'snowDepth']:
    filter_df[column] = filter_df[column].astype("float") 
filter_df["date"] = filter_df["date"].astype('datetime64[ns]')
filter_df.head()

/home/trungdc/anaconda3/envs/mast30024asm1/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,Unnamed: 0,date,tempMax,tempMin,tempAvg,tempDeparture,hdd,cdd,precipitation,newSnow,snowDepth,station
62,62,2018-01-01,19.0,8.0,13.5,-22.6,51.0,0.0,0.00,0.0,0.0,NY-LaGuardia AP Area
63,63,2018-01-02,26.0,14.0,20.0,-15.9,45.0,0.0,0.00,0.0,0.0,NY-LaGuardia AP Area
64,64,2018-01-03,30.0,17.0,23.5,-12.2,41.0,0.0,0.00,0.0,0.0,NY-LaGuardia AP Area
65,65,2018-01-04,29.0,20.0,24.5,-11.0,40.0,0.0,0.51,7.4,1.0,NY-LaGuardia AP Area
66,66,2018-01-05,20.0,10.0,15.0,-20.4,50.0,0.0,0.00,0.0,7.0,NY-LaGuardia AP Area


# Further preprocessing & Simple EDA

In [7]:
total_day = len(filter_df["date"].unique())
for station in chosen_station:
    print(f"{station} station has {len(filter_df[filter_df.station == station])}/ {total_day} days")

NY-Kennedy AP Area station has 730/ 730 days
NY-LaGuardia AP Area station has 730/ 730 days
Newark Area station has 700/ 730 days
Teterboro, NJ station has 700/ 730 days
NY-Central Park Area station has 700/ 730 days
New York JFK, NY station has 700/ 730 days


In [8]:
for column in filter_df.columns:
       print(f"Column {column} has {filter_df[column].isnull().sum()} null value")

Column Unnamed: 0 has 0 null value
Column date has 0 null value
Column tempMax has 4 null value
Column tempMin has 5 null value
Column tempAvg has 5 null value
Column tempDeparture has 5 null value
Column hdd has 5 null value
Column cdd has 5 null value
Column precipitation has 615 null value
Column newSnow has 826 null value
Column snowDepth has 764 null value
Column station has 0 null value


In [9]:
extracted_output = []
for date in filter_df["date"].unique():
    filter_by_date = filter_df[filter_df.date == date][['tempMax', 'tempMin', 'tempAvg', 'tempDeparture',
      'hdd', 'cdd', 'precipitation', 'newSnow', 'snowDepth']]
    row_output = filter_by_date.mean(skipna=True)
    row_output["date"] = date
    extracted_output.append(row_output)
extracted_output = pd.DataFrame(extracted_output)

extracted_output.head()

,tempMax,tempMin,tempAvg,tempDeparture,hdd,cdd,precipitation,newSnow,snowDepth,date
0,19.000000,6.833333,12.916667,-21.900000,51.833333,0.0,0.000000,0.00,0.0,2018-01-01
1,26.666667,12.833333,19.750000,-14.866667,44.833333,0.0,0.000000,0.00,0.0,2018-01-02
2,29.333333,12.500000,20.916667,-13.500000,44.000000,0.0,0.000000,0.00,0.0,2018-01-03
3,29.000000,19.000000,24.000000,-10.250000,40.833333,0.0,0.591667,8.32,1.0,2018-01-04
4,19.000000,9.166667,14.083333,-20.033333,50.833333,0.0,0.066667,0.00,7.4,2018-01-05


# Extracted Output storing NYC data still have na value
## Impute each value with value on previous and following day

In [10]:

for column in extracted_output.columns:
       print(f"Column {column} has {extracted_output[column].isnull().sum()} null value")

Column tempMax has 0 null value
Column tempMin has 0 null value
Column tempAvg has 0 null value
Column tempDeparture has 0 null value
Column hdd has 0 null value
Column cdd has 0 null value
Column precipitation has 13 null value
Column newSnow has 5 null value
Column snowDepth has 1 null value
Column date has 0 null value


In [11]:
for column in extracted_output.columns:
    for index, value in extracted_output[column].iteritems():
        if pd.isna(value):
            print(extracted_output.iloc[[index]])
            curdate = extracted_output.iloc[[index]]["date"].values[0]
            nextdate = curdate + np.timedelta64(1, 'D')
            predate = curdate - np.timedelta64(1, 'D')
            prevalue = extracted_output[extracted_output.date == predate][column].values[0]
            postvalue = extracted_output[extracted_output.date == nextdate][column].values[0]
            extracted_output.at[index, column] = (prevalue +postvalue)/2        

      tempMax  tempMin    tempAvg  tempDeparture   hdd  cdd  precipitation  \
28  44.833333     34.0  39.416667       6.566667  25.5  0.0            NaN   

    newSnow  snowDepth       date  
28      0.1        0.0 2018-01-29  
      tempMax    tempMin    tempAvg  tempDeparture        hdd  cdd  \
64  45.833333  30.833333  38.333333      -1.016667  26.333333  0.0   

    precipitation  newSnow  snowDepth       date  
64            NaN      0.0        0.0 2018-03-06  
     tempMax    tempMin    tempAvg  tempDeparture  hdd       cdd  \
124     76.0  59.333333  67.666667       8.466667  0.0  2.833333   

     precipitation  newSnow  snowDepth       date  
124            NaN      0.0        0.0 2018-05-05  
     tempMax  tempMin  tempAvg  tempDeparture  hdd   cdd  precipitation  \
164     85.5     69.0    77.25           5.75  0.0  12.5            NaN   

     newSnow  snowDepth       date  
164      0.0        0.0 2018-06-14  
     tempMax  tempMin  tempAvg  tempDeparture  hdd  cdd  preci

## After processing, extracteed output has No Na value

In [12]:
for column in extracted_output.columns:
       print(f"Column {column} has {extracted_output[column].isnull().sum()} null value")

Column tempMax has 0 null value
Column tempMin has 0 null value
Column tempAvg has 0 null value
Column tempDeparture has 0 null value
Column hdd has 0 null value
Column cdd has 0 null value
Column precipitation has 0 null value
Column newSnow has 0 null value
Column snowDepth has 0 null value
Column date has 0 null value


In [14]:
extracted_output.to_csv(os.path.join(data_dir, "Weather processed.csv"))